In [1]:
import os, sys
os.chdir('/home/inarighas/Projects/voice-transcript')
import pandas as pd
import numpy as np
import librosa
from bs4 import BeautifulSoup
from app.main import process_transcription

/home/inarighas/Projects/voice-transcript/.venv/lib64/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures
speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


In [2]:
root_folder = '../voice-features/eval/data/ParoleParisContemporain/'

In [3]:
import wave
import contextlib

res = []
for folder in os.listdir(root_folder)[:1]:
    if folder.startswith("record"):
        print(folder)
        list_files = os.listdir(root_folder + folder)
        for f in list_files:
            if f.endswith('.trs'):
                with open(os.path.join(root_folder, folder, f), 'tr') as tmp:
                    data = tmp.read()
                bs_data = BeautifulSoup(data, 'xml')
                speakers = bs_data.find("Speakers")
                list_speakers = [s.get('id') for s in speakers if len(s)==0]
                timestamps = {s: [] for s in list_speakers}
                for s in list_speakers:
                    for tag in bs_data.find_all('Turn', {'speaker':s}):
                        tmp = tag.attrs
                        tmp["text"] = tag.text.replace("\n", "")
                        res.append(tmp)

record-28


In [4]:

# import json
# with open(os.path.join(root_folder, folder, "transcript.json"), 'w', encoding='utf8') as f:
#     json.dump(res, f, indent=2, ensure_ascii=False)

In [6]:
import json
with open(os.path.join(root_folder, folder, "transcript.json"), 'r') as f:
    d = json.load(f)

In [8]:
d[0]

{'startTime': '0',
 'endTime': '31.210',
 'speaker': 'spk1',
 'text': "bon je reviens sur cette euh ce problème qui est un problème euh voilà de d'être chez moi combien de fois ça m'est arrivé bon ben là tu vas boulevard Voltaire c'est pas loin euh tu tu j'y vais à pieds je suis chez moi je me conditionne dans mon appartement en me disant j'y vais à pieds moi ma voiture elle est garée dans la rue j'ai un stationnement résident je passe devant je ne peux pas m'empêcher d'ouvrir euh la porte de monter dedans et et d'aller euh à euh voilà cinq minutes en voiture ce qui me mettrait peut-être euh un petit quart d'heure à pieds donc au dernier moment je prends ma voiture "}

In [10]:
# for i in np.arange(1,19):
#     name = f"PTSD_{i}"
name = os.path.join(root_folder, folder, "single_spk1.wav")
arr, sr =  librosa.load(name)
    # get list of audio(31.2*sr)]
    # list_audio = [np.array(audio[i:i + sr*60]) for i in range(0, audio.shape[0], sr*60)]
    # for idx, audio in enumerate(list_audio):
    # Run transcript for each audio subsample:
    # save retext, dur = process_transcription(arr, sr)sults


In [16]:
target_sr = 8000
speech_array = librosa.resample(arr[:int(31.2*sr)],
                                orig_sr=sr,
                                target_sr=target_sr,
                                res_type="kaiser_fast"
                                )

In [17]:
text, dur = process_transcription(speech_array, target_sr)

DEBUG:    |>>> Rel_length: tensor([1.])
DEBUG:    |>>> Signal: tensor([[ 1.1269e-08, -1.5749e-08, -2.0364e-08,  ...,  1.4285e-02,
          1.3639e-02,  6.9621e-03]])
DEBUG:    |>>> Predicted words: ['MAIS JE REVIENS SUR CE PROBLÈME QUI A UN PROBLÈME VOILÀ DETTRE CHEZ MOI COMBIEN DE FOIS ÇA MAI ARRIVÉE BON BAI LÀ TU VAS BOULEVARD DELTERRE C EST PAS LOIN J Y VAIS À PIED JE SUIS CHEZ MOI JE ME CONDITIONNE DANS MON APPARTEMENT EN DISANT J Y VAIS À PIED MOI MA VOITURE ALÉGA DANS LA RUE J AI UN STATIONNEMENT RÉSIDANT JE PASSE DEVANT JE NE PEUX PAS M EMPÊCHER D OUVRIR LA PORTE MONTER DEDAN ET D ALLER À VOILÀ À CINQ MINUTES EN VOITURE QUI ME METTRAIT PEUT ÊTRE UN PETIT QUART D HEURE À PIED DONC AU DERNIER MOMENT JE PRENDS MA VOITIE']
DEBUG:    |>>> Transcription: mais je reviens sur ce problème qui a un problème voilà dettre chez moi combien de fois ça mai arrivée bon bai là tu vas boulevard delterre c est pas loin j y vais à pied je suis chez moi je me conditionne dans mon appartement en dis

In [26]:
ref

"bon je reviens sur cette euh ce problème qui est un problème euh voilà de d'être chez moi combien de fois ça m'est arrivé bon ben là tu vas boulevard Voltaire c'est pas loin euh tu tu j'y vais à pieds je suis chez moi je me conditionne dans mon appartement en me disant j'y vais à pieds moi ma voiture elle est garée dans la rue j'ai un stationnement résident je passe devant je ne peux pas m'empêcher d'ouvrir euh la porte de monter dedans et et d'aller euh à euh voilà cinq minutes en voiture ce qui me mettrait peut-être euh un petit quart d'heure à pieds donc au dernier moment je prends ma voiture "

In [23]:
ref = d[0]['text']
hypo = text

In [25]:
import fastwer
print(f"WER: {fastwer.score_sent(hypo, ref)}")
print(f"CER: {fastwer.score_sent(hypo, ref, char_level=True)}")

WER: 39.3162
CER: 17.2859
